In [1]:
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os
import sys
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

Using TensorFlow backend.


In [2]:
classes = 8
img_row,img_col = 64,64
batch_size = 128
num_epoch = 10

num_train = 11744
num_val = 4256
train_data = "./Train"
validation_data ="./Test"

In [3]:
## data Augumentation training data ##
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 30,
    shear_range=0.3,
    zoom_range=0.4,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode="nearest")

train_generator =  train_datagen.flow_from_directory(
        train_data,
        color_mode = 'grayscale',
        target_size = (img_row,img_col),
        batch_size  = batch_size,
        class_mode = 'categorical',
        shuffle = True ) 

validation_datagen = ImageDataGenerator(
    rescale=1./255)

vaildation_generator = validation_datagen.flow_from_directory(
    validation_data,color_mode = 'grayscale',
    target_size = (img_row,img_col),
    batch_size  = batch_size,
    class_mode = 'categorical',
    shuffle = True ) 

Found 11744 images belonging to 8 classes.
Found 4256 images belonging to 8 classes.


In [4]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (img_row,img_col,1)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(8, activation = "softmax"))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
checkpoint = ModelCheckpoint(
    './base.model',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False,
    period=1
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=30,
    verbose=1,
    mode='auto'
)
tensorboard = TensorBoard(
    log_dir = './logs',
    histogram_freq=0,
    batch_size=16,
    write_graph=True,
    write_grads=True,
    write_images=False,
)

csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint,tensorboard,csvlogger,reduce]

In [6]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 96)        55392     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 96)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 96)          83040     
__________

In [8]:
########################### Train ###############################

# History = model.fit_generator(train_generator,steps_per_epoch=num_train // batch_size,validation_data=vaildation_generator,validation_steps=num_val // batch_size,epochs=num_epoch,callbacks=callbacks)

In [9]:
#model.save_weights('hand_gesture.h5')

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

plt.figure.figsize=(15,15)
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

NameError: name 'History' is not defined

In [ ]:
plt.figure.figsize=(15,15)
plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [16]:
import cv2
import numpy as np
model.load_weights('hand_gesture.h5')



In [58]:
gesture = {1:"palm",2:"L",3:"fist",4:"fist_moved",5:"thumb",6:"index_finger",7:"ok",8:"palm"}


pic = cv2.imread("/home/calsoft/Desktop/leapGestRecog/Hand_Gesture/Test/8/ok14.png")
pic = cv2.resize(pic,(64,64),1)
pic = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)

pic = np.expand_dims(np.expand_dims(pic, -1), 0)

pred = model.predict(pic)
pred = (pred.ravel()).tolist()
pred = (pred.index(1)+1)
gesture[pred]

'ok'